In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cool_maps.plot as cplt
import cartopy.crs as ccrs
import cmocean.cm as cmo
import numpy as np
from gsw import SA_from_SP, CT_from_t, rho, p_from_z, pot_rho_t_exact
import time
# import geopandas
import scipy
import datetime as dt
from collections import namedtuple
from erddapy import ERDDAP
from pprint import pprint
import multiprocessing
import matplotlib.dates as mdates
import geopandas
from cool_maps.calc import calculate_ticks, dd2dms, fmt

from data_functions import *

## Read in lease area outline shapefiles, subset to NJ lease areas and clip the glider data to those lease areas

Wind lease area outlines shape file downloaded from the [Marine Cadastre](https://hub.marinecadastre.gov/datasets/709831444a234968966667d84bcc0357_8/explore?location=33.829924%2C-96.692134%2C5.21)

In [ ]:
fname = '../Model 1/data/Wind_Lease_Outlines_2_2023.shp'
wla = geopandas.read_file(fname)
wla.head()

nj_wla=wla[wla.STATE=='New Jersey']
nj_wla

## Get all Gliders in BBOX and clip to NJ WLA

** NOTE ** This block of code takes a very long time to run

In [ ]:
Glider = namedtuple('Glider', ['name', 'lon', 'lat'])
time_formatter = '%Y-%m-%dT%H:%M:%SZ'

rename_gliders = {}
# rename_gliders["time (UTC)"] = "time"
rename_gliders["longitude"] = "lon"
rename_gliders["latitude"] = "lat"


# change stuff here
bbox=[-75,-71.5,38,41.5]

t0 = dt.datetime(2010, 1, 1, 0, 0) #start

t1 =  dt.datetime(2023, 12, 31, 0, 0)


gdf=get_active_gliders(bbox=bbox, t0=t0,t1=t1), variables=None, 
                       timeout=5, parallel=False)

gdf=gdf.reset_index()
gdf.time=pd.to_datetime(gdf.time)
gdf.to_pickle('./all_gliders_2010_present.pkl')

## Clip glider data to be inside of the NJ lease areas

In [ ]:

test = geopandas.GeoDataFrame(
    gdf, geometry=geopandas.points_from_xy(gdf.lon, gdf.lat), crs="WGS84"
)
njwla_df = geopandas.clip(test, nj_wla)
njwla_df

## Group by month and split into cold pool setup, peak, breakdown periods

In [ ]:
njwla_df=njwla_df.rename(columns={'glider':'gliders'})
njwla_df1=njwla_df.reset_index()
njwla_df1['time'].dt.month
month_group = njwla_df1.groupby(njwla_df1['time'].dt.month)
print(month_group.groups.keys())
may = month_group.get_group(5)
june = month_group.get_group(6)
july = month_group.get_group(7)
august= month_group.get_group(8)
september= month_group.get_group(9)
october= month_group.get_group(10)


set_up = pd.concat([may,june],ignore_index=True)
peak = pd.concat([july,august],ignore_index=True)
breakdown = pd.concat([september,october],ignore_index=True)

## Clean up data - get rid of profiles that min depth >3m

In [ ]:


grouped = set_up.groupby([set_up.gliders,set_up.time])
filtered_groups = [group for name, group in grouped if remove_profiles(group,thresh=3)]
set_upf = pd.concat(filtered_groups)

grouped = peak.groupby([peak.gliders,peak.time])
filtered_groups = [group for name, group in grouped if remove_profiles(group,thresh=3)]
peakf = pd.concat(filtered_groups)

grouped = breakdown.groupby([breakdown.gliders,breakdown.time])
filtered_groups = [group for name, group in grouped if remove_profiles(group,thresh=3)]
breakdownf = pd.concat(filtered_groups)

## calc density and potential density per profile per period

In [ ]:
set_up=set_upf[set_upf.columns[:-1]]
set_up = set_up.groupby([set_up.gliders,set_up.time]).apply(prof_dens)
set_up=set_up.drop(['time','gliders'],axis=1).reset_index() 
set_up = set_up.drop('level_2',axis=1)
set_up = set_up.groupby([set_up.gliders,set_up.time]).apply(prof_potdens)
set_up=set_up.drop(['time','gliders'],axis=1).reset_index()

peak=peakf[peakf.columns[:-1]]
peak = peak.groupby([peak.gliders,peak.time]).apply(prof_dens)
peak=peak.drop(['time','gliders'],axis=1).reset_index()
peak = peak.drop('level_2',axis=1)
peak = peak.groupby([peak.gliders,peak.time]).apply(prof_potdens)
peak=peak.drop(['time','gliders'],axis=1).reset_index() 

breakdown=breakdownf[breakdownf.columns[:-1]]
breakdown = breakdown.groupby([breakdown.gliders,breakdown.time]).apply(prof_dens)
breakdown=breakdown.drop(['time','gliders'],axis=1).reset_index() 
breakdown = breakdown.drop('level_2',axis=1)
breakdown = breakdown.groupby([breakdown.gliders,breakdown.time]).apply(prof_potdens)
breakdown=breakdown.drop(['time','gliders'],axis=1).reset_index() 

## Get surface and bottom depth, temperature, salinity for each profile per period

In [ ]:
setup_stuff=set_up.groupby([set_up.gliders,set_up.time]).apply(get_vars_bydepth).reset_index()   
setup_stuff[['surf_depth', 'bot_depth' , 'surf_temp', 'bot_temp', 'surf_sal', 'bot_sal','lat','lon']] = setup_stuff[0].apply(pd.Series)

peak_stuff=peak.groupby([peak.gliders,peak.time]).apply(get_vars_bydepth).reset_index()   
peak_stuff[['surf_depth', 'bot_depth' , 'surf_temp', 'bot_temp', 'surf_sal', 'bot_sal','lat','lon']] = peak_stuff[0].apply(pd.Series)

break_stuff=breakdown.groupby([breakdown.gliders,breakdown.time]).apply(get_vars_bydepth).reset_index()   
break_stuff[['surf_depth', 'bot_depth' , 'surf_temp', 'bot_temp', 'surf_sal', 'bot_sal','lat','lon']] = break_stuff[0].apply(pd.Series)

setup_stuff=setup_stuff.drop(0,axis=1)
peak_stuff=peak_stuff.drop(0,axis=1)
break_stuff=break_stuff.drop(0,axis=1)

## Calculate the upper and lower Mixed Layer Depth (MLD) and potential energy anomaly (PEA) for each profile per period

In [ ]:
mld_setup=set_up.groupby([set_up.gliders,set_up.time]).apply(calc_mld,ref_depth=4,deltaT=0.5).reset_index()
mld_setup[['mldU','mldL','qcU', 'qcL']] = mld_setup[0].apply(pd.Series)
pea_setup=set_up.groupby([set_up.gliders,set_up.time]).apply(prof_phi).reset_index().rename(columns={0:'pea'})

mld_break=breakdown.groupby([breakdown.gliders,breakdown.time]).apply(calc_mld,ref_depth=4,deltaT=0.5).reset_index()
mld_break[['mldU','mldL','qcU', 'qcL']] = mld_break[0].apply(pd.Series)
pea_break=breakdown.groupby([breakdown.gliders,breakdown.time]).apply(prof_phi).reset_index().rename(columns={0:'pea'})

mld_peak=peak.groupby([peak.gliders,peak.time]).apply(calc_mld,ref_depth=4,deltaT=0.5).reset_index()
mld_peak[['mldU','mldL','qcU', 'qcL']] = mld_peak[0].apply(pd.Series)
pea_peak=peak.groupby([peak.gliders,peak.time]).apply(prof_phi).reset_index().rename(columns={0:'pea'})

setup_stuff['pea'] = pea_setup['pea']
break_stuff['pea'] = pea_break['pea']
peak_stuff['pea'] = pea_peak['pea']
#combine with other stuff dataframes
if mld_setup['time'].equals(setup_stuff['time']):
    setup_stuff['mld_upper']=mld_setup.mldU
    setup_stuff['qc_upper']=mld_setup.qcU
    setup_stuff['mld_lower']=mld_setup.mldL
    setup_stuff['qc_lower']=mld_setup.qcL
if mld_break['time'].equals(break_stuff['time']):
    break_stuff['mld_upper']=mld_break.mldU
    break_stuff['qc_upper']=mld_break.qcU
    break_stuff['mld_lower']=mld_break.mldL
    break_stuff['qc_lower']=mld_break.qcL
if mld_peak['time'].equals(peak_stuff['time']):
    peak_stuff['mld_upper']=mld_peak.mldU
    peak_stuff['qc_upper']=mld_peak.qcU
    peak_stuff['mld_lower']=mld_peak.mldL
    peak_stuff['qc_lower']=mld_peak.qcL

## Calculate the daily average to "normalize" temporally

In [ ]:

setup_stuff=setup_stuff.groupby([setup_stuff.gliders,setup_stuff.time.dt.year.rename('year'),setup_stuff.time.dt.month.rename('month'),setup_stuff.time.dt.day.rename('day')]).mean().reset_index().drop('time',axis=1)
setup_stuff['time']=pd.to_datetime(setup_stuff[['year','month', 'day']])


peak_stuff=peak_stuff.groupby([peak_stuff.gliders,peak_stuff.time.dt.year.rename('year'),peak_stuff.time.dt.month.rename('month'),peak_stuff.time.dt.day.rename('day')]).mean().reset_index().drop('time',axis=1)
peak_stuff['time']=pd.to_datetime(peak_stuff[['year','month', 'day']])


break_stuff=break_stuff.groupby([break_stuff.gliders,break_stuff.time.dt.year.rename('year'),break_stuff.time.dt.month.rename('month'),break_stuff.time.dt.day.rename('day')]).mean().reset_index().drop('time',axis=1)
break_stuff['time']=pd.to_datetime(break_stuff[['year','month', 'day']])




## Calculate the mean, median, standard deviation of the upper and lower MLD, density, and PEA

In [ ]:

print('setup')
mld_df = setup_stuff
df = set_up
s_up,s_low,s_pp=mld_stats(mld_df,df)
print('\npeak')
mld_df = peak_stuff
df = peak
p_up,p_low,p_pp=mld_stats(mld_df,df)
print('\nbreakdown')
mld_df = break_stuff
df = breakdown
b_up,b_low,b_pp=mld_stats(mld_df,df)

## export compiste profiles that will be used for initial conditions for the models

In [ ]:
dmin=0
dmax=25
stride=0.25
vtype='avg'

# set up
df=setup_stuff
stat_up = s_up
stat_low = s_low
mldU,mldL,p_thick,dens,z,pea=create_dens_prof(df,stat_up,stat_low,dmin,dmax,stride,vtype)
dfs = pd.DataFrame({'z':z,'dens':dens.values,'mldU':[mldU]*len(z),'mldL':[mldL]*len(z),'p_thick':[p_thick]*len(z),'pea':[pea]*len(z),'per':['setup']*len(z)})

# peak
df=peak_stuff
stat_up = p_up
stat_low = p_low
mldU,mldL,p_thick,dens,z,pea=create_dens_prof(df,stat_up,stat_low,dmin,dmax,stride,vtype)
dfp = pd.DataFrame({'z':z,'dens':dens.values,'mldU':[mldU]*len(z),'mldL':[mldL]*len(z),'p_thick':[p_thick]*len(z),'pea':[pea]*len(z),'per':['peak']*len(z)})


#breakdown
df=break_stuff
stat_up = b_up
stat_low = b_low
mldU,mldL,p_thick,dens,z,pea=create_dens_prof(df,stat_up,stat_low,dmin,dmax,stride,vtype)
dfb = pd.DataFrame({'z':z,'dens':dens.values,'mldU':[mldU]*len(z),'mldL':[mldL]*len(z),'p_thick':[p_thick]*len(z),'pea':[pea]*len(z),'per':['breakdown']*len(z)})



all_df = pd.concat([dfs,dfp,dfb],ignore_index=True)
all_df.to_csv('./carp_pea_init_profs.csv',index=False)